In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

In [0]:
userdf = spark.read.csv("/FileStore/tables/userTable.csv",inferSchema=True,header=True)

In [0]:
userdf.count()

Out[3]: 34050

In [0]:
visitordf = spark.read.csv("/FileStore/tables/VisitorLogsData.csv",inferSchema=True,header=True)

In [0]:
visitordf.count()

Out[5]: 6588000

In [0]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
visitordf.createOrReplaceTempView('visit')

In [0]:
#spark.sql("select count(*) from visit v where UserID is not null ").show() #5,85,892
#spark.sql("select count(*) from visit v where VisitDateTime is not null ").show() #59,29,085
spark.sql("select count(*) from visit v where VisitDateTime is not null and UserID is not null ").show() #5,85,892

+--------+
count(1)|
+--------+
 585892|
+--------+

In [0]:
visitordf = spark.sql("""
select webClientID, visitts, to_date(visitts) visitdt, ProductID, UserID, Activity, Browser, OS, City, Country from (
                      select v.webClientID, VisitDateTime, case when instr(VisitDateTime, '-')=0 then from_unixtime(VisitDateTime/1000000000,'yyyy-MM-dd HH:mm:SS') else to_timestamp(VisitDateTime) end as visitts, v.ProductID, v.UserID, v.Activity, v.Browser, v.OS, v.City, v.Country from visit v where UserID is not null and VisitDateTime is not null)
                      
                      """)
visitordf.show(5, truncate=False)
#visitordf.printSchema()

+--------------+-----------------------+----------+---------+-------+--------+-------------+---------+-------------+-------------+
webClientID |visitts |visitdt |ProductID|UserID |Activity|Browser |OS |City |Country |
+--------------+-----------------------+----------+---------+-------+--------+-------------+---------+-------------+-------------+
WI100000112772|2018-05-15 06:48:00 |2018-05-15|Pr100017 |U106593|click |Chrome Mobile|Android | | |
WI1000009977 |2018-05-23 07:02:00 |2018-05-23|Pr101008 |U108297|null |Chrome Mobile|Android |Delhi |India |
WI100000223826|2018-05-10 06:28:53.391|2018-05-10|Pr100241 |U132443|null |Firefox |Windows |null |India |
WI10000021998 |2018-05-08 12:40:02.153|2018-05-08|pr100495 |U134616|click |Chrome |Windows |Cottage Grove|United States|
WI10000036281 |2018-05-11 15:35:43.689|2018-05-11|Pr100363 |U130784|click |Chrome |Chrome OS|New Delhi |India |
+--------------+-----------------------+----------+---------+-------+--------+-------------+---------+-------------+-------------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import col
visitordf = visitordf.withColumn("visitts", (col("visitts").cast("timestamp")))

In [0]:
visitordf.printSchema()
#visitordf.show(20,False)

root
-- webClientID: string (nullable = true)
-- visitts: timestamp (nullable = true)
-- visitdt: date (nullable = true)
-- ProductID: string (nullable = true)
-- UserID: string (nullable = true)
-- Activity: string (nullable = true)
-- Browser: string (nullable = true)
-- OS: string (nullable = true)
-- City: string (nullable = true)
-- Country: string (nullable = true)

In [0]:
visitordf.createOrReplaceTempView('visit')
userdf.createOrReplaceTempView('user')

In [0]:
visitordf.count()

Out[9]: 585892

In [0]:
xyz = spark.sql("select a.*, b.`Signup Date` SignupDate, b.`User Segment` UserSegment, max(visitdt) over (partition by 1) max_dt from visit a , user b where a.UserID=b.UserID")

In [0]:
xyz.printSchema()

root
-- webClientID: string (nullable = true)
-- visitts: timestamp (nullable = true)
-- visitdt: date (nullable = true)
-- ProductID: string (nullable = true)
-- UserID: string (nullable = true)
-- Activity: string (nullable = true)
-- Browser: string (nullable = true)
-- OS: string (nullable = true)
-- City: string (nullable = true)
-- Country: string (nullable = true)
-- SignupDate: string (nullable = true)
-- UserSegment: string (nullable = true)
-- max_dt: date (nullable = true)

In [0]:
xyz.cache()
xyz.createOrReplaceTempView('xyz')

In [0]:
spark.sql("select distinct UserID, visitdt, case when datediff(max_dt,visitdt)<= 7 then 1 else 0 end as pt from xyz where UserID = 'U106593'").show()
# spark.sql("select UserID, count(datediff(max_dt,visitdt)<= 7) as pt from xyz where UserID = 'U106593'").show()

+-------+----------+---+
 UserID| visitdt| pt|
+-------+----------+---+
U106593|2018-05-27| 1|
U106593|2018-05-25| 1|
U106593|2018-05-19| 0|
U106593|2018-05-15| 0|
U106593|2018-05-18| 0|
U106593|2018-05-16| 0|
+-------+----------+---+

In [0]:
#No_of_days_Visited_7_Days
a1 = spark.sql("""
select UserID, count(*) No_of_days_Visited_7_Days from (
select distinct UserID, visitdt  from xyz where datediff(max_dt,visitdt)<= 7 )
group by UserID
""")

In [0]:
#Pageloads_last_7_days
a2 = spark.sql("""
select UserID, Activity, count(*) Pageloads_last_7_days from (
select UserID, upper(Activity) Activity from xyz where datediff(max_dt,visitdt)<= 7 and upper(Activity) = 'PAGELOAD')
group by UserID, Activity order by UserID
""")

In [0]:
# Clicks_last_7_days
a3 = spark.sql("""
select UserID, Activity, count(*) Clicks_last_7_days from (
select UserID, upper(Activity) Activity from xyz where datediff(max_dt,visitdt)<= 7 and upper(Activity) = 'CLICK')
group by UserID, Activity order by UserID
""")

In [0]:
#No_Of_Products_Viewed_15_Days
a4 = spark.sql("""
select UserID, count(*) No_Of_Products_Viewed_15_Days from (
select distinct UserID, ProductID from xyz where datediff(max_dt,visitdt)<= 15 and ProductID is not null)
group by UserID order by UserID
""")

In [0]:
# Most_Viewed_product_15_Days
a5 = spark.sql("""
select distinct UserID ,ProductID Most_Viewed_product_15_Days from (
select *, max(visitts) over (partition by UserId) MX_DT,
count(1) over (partition by UserId) U_CNT
from (
select *, max(PRD_CNT) over (partition by UserId) MX_P_CNT
FROM (
select UserID, upper(ProductID) ProductID, visitts, visitdt, count(1) over (partition by UserId, upper(ProductID)) PRD_CNT
from xyz where datediff(max_dt,visitdt)<= 15 and ProductID is not null and upper(Activity) = 'PAGELOAD' )
) where PRD_CNT = MX_P_CNT)
where U_CNT = 1 OR (U_CNT>1 AND visitts=MX_DT)
order by 1,2

""")

In [0]:
a8.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-146261400692224> in <module> 
 ----> 1 a8 . count ( ) 

 NameError : name 'a8' is not defined

In [0]:

# User_Vintage
a6 = spark.sql("""
select distinct UserID, datediff(max_dt, SignupDate) User_Vintage from xyz """)


In [0]:
# Most_Active_OS
a7 = spark.sql("""
select UserID, OS Most_Active_OS from (
select UserID, OS, cnt, rank() over (partition by UserID order by cnt desc) rn from (
select UserID, upper(OS) OS, count(*) cnt from xyz group by UserID, upper(OS) order by UserID
)) where rn = 1
""")

In [0]:
# Recently_Viewed_Product
a8 = spark.sql("""
select distinct UserId, ProductId Recently_Viewed_Product from (
select *, max(visitts) over (partition by UserId) mx_ts from (
select UserID, upper(ProductID) ProductID, visitts
from xyz where ProductID is not null and upper(Activity) = 'PAGELOAD'
)
) where visitts = mx_ts order by 1
""")

In [0]:
a1.createOrReplaceTempView('No_of_days_Visited_7_Days')
a2.createOrReplaceTempView('Pageloads_last_7_days')
a3.createOrReplaceTempView('Clicks_last_7_days')
a4.createOrReplaceTempView('No_Of_Products_Viewed_15_Days')
a5.createOrReplaceTempView('Most_Viewed_product_15_Days')
a6.createOrReplaceTempView('User_Vintage')
a7.createOrReplaceTempView('Most_Active_OS')
a8.createOrReplaceTempView('Recently_Viewed_Product')


In [0]:
res = spark.sql("""
select UserId, No_of_days_Visited_7_Days, Pageloads_last_7_days, Clicks_last_7_days,
No_Of_Products_Viewed_15_Days, Most_Viewed_product_15_Days, User_Vintage, 
Most_Active_OS,
Recently_Viewed_Product from (
select * , rank() over (partition by UserID order by Most_Active_OS, Most_Viewed_product_15_Days, Recently_Viewed_Product) rn from (
select u.UserId, a1.No_of_days_Visited_7_Days, a2.Pageloads_last_7_days, a3.Clicks_last_7_days,
a4.No_Of_Products_Viewed_15_Days, NVL(a5.Most_Viewed_product_15_Days,'Product101') Most_Viewed_product_15_Days, a6.User_Vintage, 
a7.Most_Active_OS,
NVL(a8.Recently_Viewed_Product,'Product101') Recently_Viewed_Product
from user u
left join No_of_days_Visited_7_Days a1 on a1.UserId = u.UserId 
left join Pageloads_last_7_days a2 on a2.UserId = u.UserId
left join Clicks_last_7_days a3 on a3.UserId = u.UserId
left join No_Of_Products_Viewed_15_Days a4 on a4.UserId = u.UserId
left join Most_Viewed_product_15_Days a5 on a5.UserId = u.UserId
left join User_Vintage a6 on a6.UserId = u.UserId
left join Most_Active_OS a7 on a7.UserId = u.UserId
left join Recently_Viewed_Product a8 on a8.UserId = u.UserId)) where rn = 1
""")

In [0]:
res.repartition(1).write.format('csv').option('header','true').save("/FileStore/tables/Res4")

In [0]:
userdf.count()

Out[29]: 34050

In [0]:
x = spark.read.csv('/FileStore/tables/Res1/part-00000-tid-8965561973495206201-96aa9ec4-e23e-4759-a1f8-c32ad89287f0-27939-1-c000.csv',header='true')

In [0]:
x.count()

Out[332]: 34203

In [0]:
%fs ls /FileStore/tables/Res4/

path,name,size
dbfs:/FileStore/tables/Res4/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/tables/Res4/_committed_6482683186726281817,_committed_6482683186726281817,114
dbfs:/FileStore/tables/Res4/_started_6482683186726281817,_started_6482683186726281817,0
dbfs:/FileStore/tables/Res4/part-00000-tid-6482683186726281817-1428b2b3-5e17-4efe-b449-bb2c0c9e098f-7201-1-c000.csv,part-00000-tid-6482683186726281817-1428b2b3-5e17-4efe-b449-bb2c0c9e098f-7201-1-c000.csv,1660664


In [0]:
input_path="dbfs:/FileStore/tables/Res4/part-00000-tid-6482683186726281817-1428b2b3-5e17-4efe-b449-bb2c0c9e098f-7201-1-c000.csv"

In [0]:
in_data = spark.read.csv(input_path, header =True)

In [0]:
display(in_data)

UserId,No_of_days_Visited_7_Days,Pageloads_last_7_days,Clicks_last_7_days,No_Of_Products_Viewed_15_Days,Most_Viewed_product_15_Days,User_Vintage,Most_Active_OS,Recently_Viewed_Product
U100002,null,null,null,3,PR100258,52,ANDROID,PR100258
U100003,1,1,2,3,PR100079,1020,WINDOWS,PR100079
U100004,2,4,null,16,PR101505,340,WINDOWS,PR100069
U100005,1,1,null,3,PR100234,680,ANDROID,PR100234
U100006,1,1,null,1,PR101111,54,ANDROID,PR101111
U100007,null,null,null,1,PR100265,459,WINDOWS,PR100265
U100008,6,21,23,21,PR100184,394,ANDROID,PR100962
U100009,5,5,4,11,PR100167,77,ANDROID,PR100640
U100012,2,6,14,8,PR100055,123,MAC OS X,PR100055
U100013,3,5,3,5,PR100177,1686,MAC OS X,PR100134
